# Integración de Datos Datawarehouse

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hermetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd '/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module'

/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module


## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein
import re

# Ruta de archivos

In [2]:
base_path = os.getcwd()
path = os.path.join(base_path, "CLEANED_DATA")
path_datawarehouse = os.path.join(base_path, "DATAWAREHOUSE")
path_datamart_oc = 'datamart_oc'
path_datamart_oec = 'datamart_oec'
path_datamart_ci = 'datamart_ci'

 ## Tabla Ubicacion

In [3]:
df_ubicacion = pd.read_csv(os.path.join(path, 'ubicacion.csv'))

# Funciones de procesamiento de datos

In [4]:
year_list = [str(x) for x in range(2022,2009,-1)]

def get_year(x):
    for year in year_list:
        if re.search('-'+year+'\s|'+year+'-|-'+year+'$',x):
            return year
    return 'sin_anio'

In [5]:
def get_year_by_oc(x,y):
    if x=='sin_anio':
        if y != 'sin fecha resolución' and not pd.isna(y):
            return y[-4:]
    else:
        return x

# Datamart OC

## Dimensión Razon Social OC

In [25]:
df_oc = pd.read_csv(os.path.join(path,'oc.csv'))
df_oc_ug = pd.read_csv(os.path.join(path,'oc_dl_provincia_canton.csv'))
df_oc_ug['estado']='desconocido'
df_razon_social_oc = pd.concat([df_oc[['razon_social','estado','canton']],df_oc_ug[['razon_social','estado','canton']]],axis=0)
df_razon_social_oc = df_razon_social_oc.drop_duplicates(subset=['razon_social','canton'],keep='first')
df_razon_social_oc = pd.merge(df_razon_social_oc,df_ubicacion,on='canton',how="left")
df_razon_social_oc.pop('id_ubicacion')
df_razon_social_oc = df_razon_social_oc.fillna({'provincia':'sin provincia'})
df_razon_social_oc = df_razon_social_oc.rename_axis('id_oc').reset_index().astype('object')
#df_razon_social_oc = df_razon_social_oc.apply(lambda x: x.astype(str).str.lower())

In [26]:
#df_razon_social_oc.head()
#df_razon_social_oc.info()

In [27]:
df_razon_social_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_razon_social_oc.csv'),index=False)

## Dimensión Curso_Perfil

In [28]:
df_oc_cl = pd.read_csv(os.path.join(path,'oc_dl_familia_sector_perfil.csv'))
df_oc_cl_2 = pd.read_csv(os.path.join(path,'oc_cl.csv'))
df_oc_cl_2.pop('ruc_o_codigo')
df_oc_cl = pd.concat([df_oc_cl,df_oc_cl_2],axis=0)

In [29]:
df_oc_cl = df_oc_cl.drop_duplicates()
df_oc_cl['modalidad'] = 'desconocida'
df_oc_cl['carga_horaria'] = 0
df_oc_cl['tipo'] = 'perfil'

In [30]:
df_oc_cc = pd.read_csv(os.path.join(path,'oc_cc.csv'))
df_oc_cc.pop('documento')
df_oc_cc['tipo'] = 'curso'

In [31]:
data_curso_perfil = np.concatenate((df_oc_cc.values,df_oc_cl.values), axis=0)
df_curso_perfil = pd.DataFrame(data=data_curso_perfil,columns=['razon_social','area_familia','especialidad_sector','curso_perfil','modalidad','carga_horaria','tipo'])
df_curso_perfil.insert(0,'tipo',df_curso_perfil.pop('tipo'))
df_curso_perfil = df_curso_perfil.rename_axis('id_curso_perfil').reset_index().astype('object')
df_curso_perfil = df_curso_perfil.apply(lambda x: x.astype(str).str.lower())
df_curso_perfil['carga_horaria'] = df_curso_perfil['carga_horaria'].astype('float64')

In [32]:
#df_curso_perfil.head()
#df_curso_perfil.info()

## Hechos OC

In [33]:
#AGREGAR AÑO
# Cargar los archivos necesarios de la ubicación de CLEANED DATA
df_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))
df_pcoc = df_pcoc.apply(lambda x: x.astype(str).str.lower())

# Se extraera el año de la columna Numero_certificado.
# Para esto se buscará porciones de texto que contengan los años del 2000 - 2022 (año actual)

df_pcoc['anio'] = df_pcoc['numero_certificado'].apply(get_year)

df_oc = pd.read_csv(os.path.join(path,'oc.csv'))

df_pcoc = pd.merge(df_pcoc,df_oc[['razon_social','fecha_resolucion']],left_on=['razon_social_oc'],right_on=['razon_social'],how="left") 


df_pcoc['anio'] = df_pcoc.apply(lambda x: get_year_by_oc(x['anio'], x['fecha_resolucion']), axis=1)

df_pcoc[
    (df_pcoc['anio']=='sin_anio') & 
    (df_pcoc['fecha_resolucion']!='sin fecha resolución') &
    (df_pcoc['fecha_resolucion'].notnull())
    ]['anio'] = df_pcoc[
    (df_pcoc['anio']=='sin_anio') & 
    (df_pcoc['fecha_resolucion']!='sin fecha resolución') &
    (df_pcoc['fecha_resolucion'].notnull())
    ]['fecha_resolucion'].str.slice(start=-4)

df_pcoc = df_pcoc.drop(['razon_social','fecha_resolucion'],axis=1)


## Dimension Fecha OC

Se debe desarrollar en base a el año el cual se haya obtenido el certificado del curso o perfil OC.

En caso de no existir un año registrado en el certificado en la tabla **PCOC** se deberá tomar como referencia el año de resolución de la Razon social de la tabla **OC**

In [34]:
df_fecha_oc = pd.DataFrame(data=df_pcoc['anio'].sort_values().unique(),columns=['anio'])
df_fecha_oc = df_fecha_oc.rename_axis('id_fecha').reset_index().astype('str')
df_fecha_oc = df_fecha_oc[:-1]

In [35]:
df_fecha_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_fecha.csv'),index=False)

## Guardar Datamart OC

In [36]:
## UNIR DATOS
df_fact_oc = df_pcoc.groupby(['razon_social_oc','nombre_curso_perfil','anio']).count().reset_index().iloc[:,:4]
df_fact_oc = df_fact_oc.rename(columns={'numero_documento':'num_cap_cer'})
df_total_cap_oc = df_pcoc.groupby(['razon_social_oc','anio']).count().reset_index().iloc[:,:3]
df_total_cap_oc = df_total_cap_oc.rename(columns={'numero_documento':'volumen_capacitados'})
df_total_cur_oc = df_pcoc.groupby(['razon_social_oc','nombre_curso_perfil','anio']).count().reset_index()
df_total_cur_oc = df_total_cur_oc.groupby(['razon_social_oc','anio']).count().reset_index().iloc[:,:3]
df_total_cur_oc = df_total_cur_oc.rename(columns={'nombre_curso_perfil':'total_cursos'})
df_fact_oc = pd.merge(df_fact_oc,df_razon_social_oc,left_on=['razon_social_oc'],right_on=['razon_social'],how="inner")
df_fact_oc = pd.merge(df_fact_oc,df_curso_perfil,left_on=['razon_social_oc','nombre_curso_perfil'],right_on=['razon_social','curso_perfil'],how="inner")
df_fact_oc = pd.merge(df_fact_oc,df_fecha_oc,left_on=['anio'],right_on=['anio'],how="inner")

In [37]:
df_fact_oc = pd.merge(df_fact_oc,df_total_cap_oc,on=['razon_social_oc','anio'],how="inner")
df_fact_oc = pd.merge(df_fact_oc,df_total_cur_oc,on=['razon_social_oc','anio'],how="inner")

In [38]:
df_fact_oc = df_fact_oc[['id_oc','id_curso_perfil','id_fecha','num_cap_cer','carga_horaria','total_cursos','volumen_capacitados']]
df_fact_oc['total_horas'] = df_fact_oc['num_cap_cer']*df_fact_oc['carga_horaria']
df_fact_oc.drop('carga_horaria',axis=1,inplace=True)

In [39]:
df_fact_oc['total_horas'] = df_fact_oc['total_horas'].astype('int64')


In [40]:
#df_fact_oc.head()
#df_fact_oc.info()

In [41]:
df_curso_perfil.drop('razon_social',axis=1,inplace=True)

In [42]:
df_curso_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_curso_perfil.csv'),index=False)
df_fact_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'fact_oc.csv'),index=False)

# Dataframe OEC

### Dimensión Razon Social OEC

In [93]:
df_oec_2 = pd.read_csv(os.path.join(path,'oec_dl_provincia_canton.csv'))
df_oec_2['estado'] = 'desconocido'
df_oec_2 = df_oec_2[['razon_social','estado','canton']]
df_oec = pd.read_csv(os.path.join(path,'oec.csv'))
df_razon_social_oec = df_oec[['razon_social','estado','canton']]
df_razon_social_oec = pd.concat([df_razon_social_oec,df_oec_2])
df_razon_social_oec = pd.merge(df_razon_social_oec,df_ubicacion,on='canton',how="left")
df_razon_social_oec.pop('id_ubicacion')
df_razon_social_oec = df_razon_social_oec.fillna({'provincia':'sin provincia'})
df_razon_social_oec = df_razon_social_oec.drop_duplicates(subset=['razon_social'], keep='first')
df_razon_social_oec = df_razon_social_oec.rename_axis('id_oec').reset_index().astype('object')
df_razon_social_oec = df_razon_social_oec.apply(lambda x: x.astype(str).str.lower())

In [37]:
#df_razon_social_oec.head()
#df_razon_social_oec.info()

In [94]:
df_razon_social_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_razon_social_oec.csv'),index=False)

### Dimension Perfil

In [95]:
df_perfil = pd.read_csv(os.path.join(path,'oec_dl_familia_sector_perfil.csv'))
df_perfil = df_perfil.rename(columns={'Nombre':'razon_social','Familia':'familia','Sector':'sector','Perfil':'perfil'})
df_perfil = df_perfil.rename_axis('id_perfil').reset_index().astype('object')
df_perfil['modalidad'] = 'desconocida'
df_perfil = df_perfil.apply(lambda x: x.astype(str).str.lower())
#df_perfil['carga_horaria'] = df_perfil['carga_horaria'].astype('float64')
df_perfil['carga_horaria'] = 6
df_perfil['costo'] = 250.0

In [96]:
#df_perfil.head()
#df_perfil.info()

In [97]:
df_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_perfil.csv'),index=False)

## Hechos OEC

In [98]:
df_pcoec = pd.read_csv(os.path.join(path,'pcoec.csv'))
df_demanda_oec = df_pcoec.groupby(['razon_social','perfil','fecha_certificacion']).count().reset_index().iloc[:,:4]

## Dimension Fecha

In [100]:
df_fecha_oec = df_pcoec[['fecha_certificacion']].groupby(['fecha_certificacion']).count().reset_index()
data_fecha = df_fecha_oec['fecha_certificacion'].str.rsplit("-", expand=True)

In [101]:
df_fecha_oec = pd.concat([df_fecha_oec,data_fecha],axis=1)
df_fecha_oec = df_fecha_oec.rename(columns={0:'dia',1:'mes',2:'anio'})
df_fecha_oec = df_fecha_oec.rename_axis('id_fecha').reset_index().astype('str')

In [102]:
#df_fecha_oec.head()
#df_fecha_oec.info()

In [103]:
df_demanda_oec = df_demanda_oec.rename(columns={'apellidos_nombres':'num_cer'})
df_total_cap_oec = df_pcoec.groupby(['razon_social','fecha_certificacion']).count().reset_index()
df_total_cap_oec = df_total_cap_oec.rename(columns={'numero_certificacion':'volumen_capacitados'})
df_total_cap_oec = df_total_cap_oec[['razon_social','fecha_certificacion','volumen_capacitados']]
df_total_cur_oec = df_pcoec.groupby(['razon_social','perfil','fecha_certificacion']).count().reset_index()
df_total_cur_oec = df_total_cur_oec.groupby(['razon_social','fecha_certificacion']).count().reset_index()
df_total_cur_oec = df_total_cur_oec.rename(columns={'numero_certificacion':'total_cursos'})
df_total_cur_oec = df_total_cur_oec[['razon_social','fecha_certificacion','total_cursos']]
df_demanda_oec = pd.merge(df_demanda_oec,df_total_cap_oec,on=['razon_social','fecha_certificacion'],how="inner")
df_demanda_oec = pd.merge(df_demanda_oec,df_total_cur_oec,on=['razon_social','fecha_certificacion'],how="inner")
df_demanda_oec = pd.merge(df_demanda_oec,df_razon_social_oec,on='razon_social',how="inner")
df_demanda_oec = pd.merge(df_demanda_oec,df_perfil,on=['razon_social','perfil'],how="inner")
df_demanda_oec = pd.merge(df_demanda_oec,df_fecha_oec,on='fecha_certificacion',how="inner")
df_demanda_oec['total_horas'] = df_demanda_oec['num_cer']*df_demanda_oec['carga_horaria']
df_demanda_oec = df_demanda_oec.rename(columns={'costo':'total_ganacias'})
df_demanda_oec = df_demanda_oec[['id_oec','id_perfil','id_fecha','num_cer','total_horas','total_cursos','volumen_capacitados','total_ganacias']]


In [104]:
#df_demanda_oec.head()
#df_demanda_oec.info()

In [105]:
df_fecha_oec = df_fecha_oec.drop('fecha_certificacion',axis=1)
df_fecha_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_fecha.csv'),index=False)

In [106]:
df_perfil = df_perfil.drop('razon_social',axis=1)
df_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_perfil.csv'),index=False)

In [107]:
df_demanda_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'fact_oec.csv'),index=False)

# Dataframe CI

## Dimensión Capacitador independiente

In [50]:
df_capacitador_ci = pd.read_csv(os.path.join(path,'ci.csv'))
df_capacitador_ci = df_capacitador_ci[['razon_social','codigo_resolucion']]
df_capacitador_ci_2 = pd.read_csv(os.path.join(path,'ci_dl_provincia_canton.csv'))
df_capacitador_ci = pd.merge(df_capacitador_ci,df_capacitador_ci_2,left_on=['razon_social'],right_on=['razon_social'],how="inner")
df_capacitador_ci_2.insert(1,'codigo_resolucion','desconocido')
df_capacitador_ci = pd.DataFrame(data=np.concatenate((df_capacitador_ci.values,df_capacitador_ci_2.values), axis=0),columns=['razon_social','codigo_resolucion','provincia','canton'])
df_capacitador_ci = df_capacitador_ci.drop_duplicates(subset=['razon_social','provincia','canton'],keep='first')
df_capacitador_ci = df_capacitador_ci.fillna({'provincia':'sin provincia','canton':'sin canton'})
df_capacitador_ci['anio'] = df_capacitador_ci['codigo_resolucion'].apply(get_year)
df_capacitador_ci  = df_capacitador_ci.reset_index(drop=True)
df_capacitador_ci  = df_capacitador_ci.rename_axis('id_ci').reset_index().astype('str')
df_capacitador_ci = df_capacitador_ci.apply(lambda x: x.astype(str).str.lower())

In [51]:
#df_capacitador_ci.tail()
#df_capacitador_ci.info()

### Dimesión Fecha CI

In [52]:
df_fecha_ci = pd.DataFrame(data=df_capacitador_ci['anio'].unique(),columns=['anio'])
df_fecha_ci = df_fecha_ci.rename_axis('id_fecha').reset_index().astype('str')

## Dimensión Curso

In [53]:
df_ci_lca = pd.read_csv(os.path.join(path,'ci_lca.csv'))
df_ci_lca =  df_ci_lca.rename_axis('id_curso').reset_index().astype('str')
df_ci_lca["carga_horaria"]=df_ci_lca["carga_horaria"].astype('int64')
df_ci_lca.insert(6,'modalidad', 'desconocida')

In [42]:
#df_ci_lca.tail()
# df_ci_lca.info()

## Hechos CI

In [58]:
df_pcci = pd.read_csv(os.path.join(path,'pcci.csv'))
df_pcci = pd.merge(df_pcci,df_capacitador_ci[['anio','razon_social']],on=['razon_social'],how="inner")
df_pcci = df_pcci.rename(columns={'numero_documento':'num_cap'})
#df_capacitador_ci = df_capacitador_ci.drop(['anio'],axis=1)
fact_ci = df_pcci.groupby(['razon_social','curso','anio']).count().reset_index()
fact_ci = pd.merge(fact_ci,df_capacitador_ci[['razon_social','id_ci']],on=['razon_social'],how="inner")

In [59]:
fact_ci.head()

,razon_social,curso,anio,num_cap,apellidos_nombres,numero_horas,id_ci
0,abata carrera gloria beatriz,curso básico de la competencia lingüística del...,2021,14,14,14,184
1,abril velez erika lucia,primeros auxilios.,2022,107,107,107,869
2,acosta mensias david alejandro,gestión de los procesos de contratación en la ...,2021,69,69,69,543
3,acosta mensias luis eduardo,administración estratégica de subsistemas de r...,2020,6,6,6,61
4,acosta mensias luis eduardo,auditoría interna bajo la norma iso 9001,2020,6,6,6,61


In [60]:
df_pcci = pd.read_csv(os.path.join(path,'pcci.csv'))
df_pcci = pd.merge(df_pcci,df_capacitador_ci[['anio','razon_social']],on=['razon_social'],how="inner")
df_pcci = df_pcci.rename(columns={'numero_documento':'num_cap'})

fact_ci = df_pcci.groupby(['razon_social','curso','anio']).count().reset_index()
fact_ci = pd.merge(fact_ci,df_capacitador_ci[['razon_social','id_ci']],on=['razon_social'],how="inner")
df_total_cap_ci = df_pcci.groupby(['razon_social','anio']).count().reset_index()
df_total_cap_ci = df_total_cap_ci.rename(columns={'num_cap':'volumen_capacitados'})
df_total_cap_ci = df_total_cap_ci[['razon_social','anio','volumen_capacitados']]
df_total_cur_ci = df_pcci.groupby(['razon_social','curso','anio']).count().reset_index()
df_total_cur_ci = df_total_cur_ci.groupby(['razon_social','anio']).count().reset_index()
df_total_cur_ci = df_total_cur_ci.rename(columns={'num_cap':'total_cursos'})
df_total_cur_ci = df_total_cur_ci[['razon_social','anio','total_cursos']]
fact_ci = pd.merge(fact_ci,df_total_cap_ci,on=['razon_social','anio'],how="inner")
fact_ci = pd.merge(fact_ci,df_total_cur_ci,on=['razon_social','anio'],how="inner")
fact_ci = pd.merge(fact_ci,df_ci_lca,on=['razon_social','curso'],how="inner")
fact_ci = pd.merge(fact_ci,df_fecha_ci,left_on=['anio'],right_on=['anio'],how="inner")
fact_ci['num_cap'] = fact_ci['num_cap'].astype('int64')
fact_ci['total_horas'] = fact_ci['num_cap']*fact_ci['carga_horaria']
fact_ci['total_horas'] = fact_ci['total_horas'].astype('int64')
fact_ci = fact_ci[['id_ci','id_curso','id_fecha','num_cap','total_horas','total_cursos','volumen_capacitados','carga_horaria']]

In [64]:
#fact_ci.tail()
#fact_ci.info()
# fact_ci.describe(include='all')

### Guardar Datamart

In [63]:
df_ci_lca = df_ci_lca[['id_curso', 'curso', 'area','especialidad', 'modalidad', 'carga_horaria']]
df_ci_lca.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_curso_ci.csv'),index=False)

In [44]:
df_capacitador_ci = df_capacitador_ci.drop(['anio'],axis=1)
df_capacitador_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_razon_social_ci.csv'),index=False)
df_fecha_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_fecha_ci.csv'),index=False)
fact_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'fact_ci.csv'),index=False)

# Costos

In [45]:
lev = Levenshtein()

In [46]:
stopwords = open(os.path.join(path,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]

In [47]:
def normalizar(text):
    puntuacion = '!"#$%&:;\'()*+,-./?@[\]^_`{|}~()'
    text = text.lower() 
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.strip()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords:
            new_text.append(text[i])
    new_text = ' '.join(new_text)
            
    return new_text

In [48]:
#Filtrar las areas de Tecnologia y TICs
df_dim_curso_perfil = pd.read_csv(os.path.join(path_datawarehouse,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[(df_dim_curso_perfil['area_familia']=='información y comunicación')|(df_dim_curso_perfil['area_familia'] == 'tecnologías de la información y comunicación')]
data_1= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data_1 = data_1[['curso_perfil','modalidad']]

'''
df_dim_perfil = pd.read_csv(os.path.join(path_datawarehouse,'datamart_oec','dim_perfil.csv'))
df_dim_perfil = df_dim_perfil[df_dim_perfil['familia']=='tecnología: hardware y software (incluye tic)']
data_2 = df_dim_perfil.groupby('perfil').count().reset_index()
data_2['modalidad'] = 'desconocida'
data_2 = data_2[['perfil','modalidad']]
'''

df_dim_curso = pd.read_csv(os.path.join(path_datawarehouse,'datamart_ci','dim_curso_ci.csv'))
df_dim_curso = df_dim_curso[df_dim_curso['area']=='tecnologías de la información y comunicación']
data_3 = df_dim_curso.groupby('curso').count().reset_index()
data_3['modalidad'] = 'desconocida'
data_3 = data_3[['curso','modalidad']]

arr_dims_curso_perfil = [df_dim_curso_perfil,df_dim_curso]

In [49]:
data = np.concatenate((data_1.values,data_3.values),axis=0)
data = pd.DataFrame(data=data,columns=['curso_perfil','modalidad'])
data = data.drop_duplicates()
data['aux_2_curso_perfil'] = data['curso_perfil'].str.cat(data['modalidad'],sep=' | ')
data['aux_curso_perfil'] = data['curso_perfil'].apply(lambda x: normalizar(x))

In [50]:
df_coec = pd.read_csv(os.path.join(path,'costos_estimados_cursos.csv'))
df_coec['aux_2_curso'] = df_coec['curso'].str.cat(df_coec['modalidad'],sep=' | ')
df_coec['aux_curso'] = df_coec['curso'].apply(lambda x: normalizar(x))

In [51]:
arr = []

for i in range(len(df_coec['aux_curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['aux_curso_perfil'])):
        curso_fuente = df_coec['aux_curso'].iloc[i]
        curso_comparar = data['aux_curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar,(1,1.5,1.25))
        aux_similary=0
        arr_curso = curso_comparar.split(' ')
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
            if data['modalidad'].iloc[j] == df_coec['modalidad'].iloc[i]:
                aux_similary += aux_similary/2
        if aux_similary > similary and aux_similary > 2/len(arr_curso):
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [52]:
res = pd.DataFrame(data=arr,index=df_coec['aux_2_curso'],columns=data['aux_2_curso_perfil'])

In [66]:
arr_dims_cols = ['curso_perfil','curso']
arr_datamarts = ['datamart_oc','datamart_ci']
arr_files = ['dim_curso_perfil.csv','dim_curso_ci.csv']
arr_df_final_dims = []
for i in range(len(arr_dims_curso_perfil)):
  df_dim = arr_dims_curso_perfil[i]
  df_dim['curso_modalidad'] = df_dim[arr_dims_cols[i]].str.cat(df_dim['modalidad'],sep=' | ')
  arr_curso_fuente = []
  for j in range(arr_dims_curso_perfil[i].shape[0]):
      query = df_dim['curso_modalidad'].iloc[j]
      val = res[res[query]==res[query].max()][query].index.values[0]
      arr_curso_fuente.append(df_coec[df_coec['aux_2_curso']==val][['curso','precio_por_hora']].values[0])
  df_curso_fuente_precio = pd.DataFrame(data=arr_curso_fuente,columns=['curso_fuente','precio_por_hora'])
  df_res_corr = pd.DataFrame(data=[df_dim[arr_dims_cols[i]].values,df_dim['modalidad'].values,df_curso_fuente_precio['curso_fuente'].values,df_curso_fuente_precio['precio_por_hora'].values])
  df_res_corr = df_res_corr.transpose()
  df_res_corr = df_res_corr.rename(columns={0:arr_dims_cols[i],1:'modalidad',2:'curso_fuente',3:'precio_por_hora'})
  df_dim = pd.read_csv(os.path.join(path_datawarehouse,arr_datamarts[i],arr_files[i]))
  df_dim= pd.merge(df_dim,df_res_corr,left_on=[arr_dims_cols[i],'modalidad'],right_on=[arr_dims_cols[i],'modalidad'],how='left')
  df_dim['costo'] = df_dim['carga_horaria']*df_dim['precio_por_hora']
  df_dim = df_dim.drop(['curso_fuente','precio_por_hora'], axis=1)
  df_dim = df_dim.drop_duplicates()
  df_dim.to_csv(os.path.join(path_datawarehouse,arr_datamarts[i],arr_files[i]),index=False)
  arr_df_final_dims.append(df_dim)


## Agregar Total precio

In [54]:
df_dim_curso_perfil = pd.read_csv(os.path.join(path_datawarehouse,'datamart_oc','dim_curso_perfil.csv'))
df_fact_oc = pd.read_csv(os.path.join(path_datawarehouse,path_datamart_oc,'fact_oc.csv'))
df_fact_oc_mod = pd.merge(df_fact_oc,df_dim_curso_perfil,on='id_curso_perfil',how='inner')
df_fact_oc_mod['total_ganancias'] = df_fact_oc_mod['num_cap_cer']*df_fact_oc_mod['costo']
df_fact_oc_mod_res = df_fact_oc_mod[['id_oc','id_curso_perfil','id_fecha','num_cap_cer','total_cursos','volumen_capacitados','total_horas','total_ganancias']]

In [67]:
df_dim_curso = pd.read_csv(os.path.join(path_datawarehouse,'datamart_ci','dim_curso_ci.csv'))
df_fact_ci = pd.read_csv(os.path.join(path_datawarehouse,path_datamart_ci,'fact_ci.csv'))
df_fact_ci_mod = pd.merge(df_fact_ci,df_dim_curso,on='id_curso',how='inner')
df_fact_ci_mod['total_ganancias'] = df_fact_ci_mod['num_cap']*df_fact_ci_mod['costo']
df_fact_ci_mod_res = df_fact_ci_mod[['id_ci','id_curso','id_fecha','num_cap','total_cursos','volumen_capacitados','total_horas','total_ganancias']]


In [68]:
df_fact_ci_mod_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 1099
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_ci                1100 non-null   int64  
 1   id_curso             1100 non-null   int64  
 2   id_fecha             1100 non-null   int64  
 3   num_cap              1100 non-null   int64  
 4   total_cursos         1100 non-null   int64  
 5   volumen_capacitados  1100 non-null   int64  
 6   total_horas          1100 non-null   int64  
 7   total_ganancias      63 non-null     float64
dtypes: float64(1), int64(7)
memory usage: 77.3 KB


In [57]:
df_fact_oc_mod_res = df_fact_oc_mod_res.fillna({'total_ganancias':0})
df_fact_ci_mod_res = df_fact_ci_mod_res.fillna({'total_ganancias':0})

In [58]:
df_fact_oc_mod_res.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'fact_oc.csv'),index=False)
df_fact_ci_mod_res.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'fact_ci.csv'),index=False)